In [3]:
import requests
import json
import jsbeautifier
from bs4 import BeautifulSoup
from pathlib import Path

In [7]:
verb = 'reeixir'

In [8]:
path = Path() / 'data' / 'raw' 
path.mkdir(parents=True, exist_ok=True)
file_path = path / f'{verb}.json'

In [9]:
if not file_path.exists():
    print(f'Fetching data from diccionari.cat, parsing and writing it to {path} ...')
    response = requests.get(f'https://www.diccionari.cat/conjugador_catala/{verb}')
    soup = BeautifulSoup(response.text, 'html.parser')
    root = soup.find(string='INFINITIU').find_parent('ol').find_parent('ol')
    data = {'gerundi': root.find(string='GERUNDI').find_parent('ol').find('li').text}
    categorias = ['singlar masculí', 'singular femení', 'plural masculí', 'plural femení' ]
    data['participi'] = dict(zip(categorias, [x.text for x in root.find(string='PARTICIPI').find_parent('ol').find_all('li')]))
    personas = ['jo', 'tu', 'ell, ella, vosté', 'nosaltres', 'vosaltres', 'ells, elles, vostés']
    indicatiu = root.find(string='INDICATIU').find_parent('ol')
    present = dict(zip(personas, [x.text for x in indicatiu.find(string='PRESENT').find_parent('ol').find_all('li')]))
    imperfet = dict(zip(personas, [x.text for x in indicatiu.find(string='IMPERFET').find_parent('ol').find_all('li')]))
    passat = dict(zip(personas, [x.text for x in indicatiu.find(string='PASSAT').find_parent('ol').find_all('li')]))
    futur = dict(zip(personas, [x.text for x in indicatiu.find(string='FUTUR').find_parent('ol').find_all('li')]))
    condicional = dict(zip(personas, [x.text for x in indicatiu.find(string='CONDICIONAL').find_parent('ol').find_all('li')]))
    data['indicatiu'] = {'present': present, 'imperfet': imperfet, 'passat': passat, 'futur': futur, 'condicional': condicional}
    subjuntiu = root.find(string='SUBJUNTIU').find_parent('ol')
    present = dict(zip(personas, [x.text for x in subjuntiu.find(string='PRESENT').find_parent('ol').find_all('li')]))
    imperfet = dict(zip(personas, [x.text for x in subjuntiu.find(string='IMPERFET').find_parent('ol').find_all('li')]))
    data['subjuntiu'] = {'present': present, 'imperfet': imperfet}
    personas_imperativas = ['tu', 'ell, ella, vosté', 'nosaltres', 'vosaltres', 'ells, elles, vostés']
    data['imperatiu'] = dict(zip(personas_imperativas, [x.text for x in root.find(string='IMPERATIU').find_parent('ol').find_all('li')]))
    opts = jsbeautifier.default_options()
    opts.indent_size = 2
    with open(file_path, 'w') as fp:
        fp.write(jsbeautifier.beautify(json.dumps(data), opts))
else:
    print(f'File {file_path} already exists. Skipping ...')

Fetching data from diccionari.cat, parsing and writing it to data/raw ...
